In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import pickle
import tensorflow as tf
import re
from collections import Counter
import math
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# -------------------------
# 1️⃣ โหลด Scaler, Tokenizer, LabelEncoder
# -------------------------
with open("utils/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("utils/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("utils/labelencoder.pkl", "rb") as f:
    le = pickle.load(f)

maxlen = 200  # ต้องตรงกับตอน train
# -------------------------
# 2️⃣ กำหนด Custom Attention Layer
# -------------------------
@tf.keras.utils.register_keras_serializable()
class Attention(Layer):
    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(shape=(input_shape[-1],), initializer='zeros', trainable=True)
        self.u = self.add_weight(shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)

    def call(self, x):
        u_it = tf.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        a_it = tf.nn.softmax(tf.tensordot(u_it, self.u, axes=1), axis=1)
        return tf.reduce_sum(x * a_it, axis=1)

# -------------------------
# 3️⃣ โหลดโมเดลพร้อม custom_objects
# -------------------------
model = load_model("utils/model.keras", custom_objects={"Attention": Attention})

# -------------------------
# 4️⃣ ฟังก์ชัน extract_url_features (ต้องมีฟังก์ชันช่วยเหลือด้านล่างด้วย)

# -------------------------

BRAND_KEYWORDS = [
    "paypal","apple","amazon","bank","chase","facebook","meta","google","microsoft",
    "outlook","office365","instagram","line","kbank","scb","krungsri","kplus"
]

COMMON_TLDS = set([
 "com","net","org","info","biz","co","io","ai","app","edu","gov","mil","ru","de","uk","cn","fr","jp","br","in","it","es","au","nl","se","no"
])

from urllib.parse import urlparse

def parse_host_and_scheme(url: str): #ดึง host and scheme (http/https).
    p = urlparse(url)
    host = (p.hostname or "").lower()
    scheme = (p.scheme or "").lower()
    return host, scheme

def is_ip_host(host: str): #if host is IP = 1.
    return bool(re.fullmatch(r"(?:\d{1,3}\.){3}\d{1,3}", host))

def count_subdomains(host: str): #Counting subdomains.
    if not host: return 0
    return max(0, len(host.split(".")) - 2)

def has_double_slash_in_path(url: str): #Check double slash "//" in the path.
    return "//" in (urlparse(url).path or "")

def has_tld_in_path(url: str): #Check TLD in path.
    path = (urlparse(url).path or "").lower()
    return any(("."+tld) in path for tld in COMMON_TLDS)

def has_symbols_in_domain(host: str): #Check symbol in domain.
    return bool(re.search(r"[^a-z0-9\.-]", host))

def domain_prefix_suffix_like_brand(host: str): #Check pattern where prefix/suffix might mimic brand.
    if not host: return False
    first = host.split(".")[0]
    return any(b in first and "-" in first for b in BRAND_KEYWORDS)

def brand_in_path_or_subdomain(host: str, url: str): #Check brand in path or subdomains.
    text = (host + " " + urlparse(url).path + " " + urlparse(url).query).lower()
    return any(b in text for b in BRAND_KEYWORDS)

def digit_count_in_domain(host: str): #Check digit in domain
    return sum(c.isdigit() for c in host)

def url_entropy(url: str): #Check ความซับซ้อนของ url
    if not url:
        return 0.0
    counts = Counter(url)
    total = len(url)
    entropy = -sum((count/total) * math.log2(count/total) for count in counts.values())
    return entropy

def extract_url_features(url: str):
    host, scheme = parse_host_and_scheme(url)
    return {
        "is_ip_host": int(is_ip_host(host)),
        "subdomain_count": count_subdomains(host),
        "double_slash_in_path": int(has_double_slash_in_path(url)),
        "tld_in_path": int(has_tld_in_path(url)),
        "symbols_in_domain": int(has_symbols_in_domain(host)),
        "prefix_suffix_like_brand": int(domain_prefix_suffix_like_brand(host)),
        "brand_in_path_or_subdomain": int(brand_in_path_or_subdomain(host, url)),
        "url_length": len(url),
        "scheme_https": 1 if scheme == "https" else 0,
        "digit_count_domain": digit_count_in_domain(host),
        "url_entropy": url_entropy(url)
    }

# -------------------------
# 5️⃣ ฟังก์ชัน predict
# -------------------------
def predict_url(url):
    # แปลง structured features
    struct_feat = scaler.transform([list(extract_url_features(url).values())])
    # แปลง text → sequence
    seq = pad_sequences(tokenizer.texts_to_sequences([url]), maxlen=maxlen)

    # predict
    pred = model.predict([seq, struct_feat])[0]
    # convert index → label
    label = le.inverse_transform([np.argmax(pred)])[0]
    return label, pred

# -------------------------
# 6️⃣ ตัวอย่างใช้งาน
# -------------------------
url_test = "http://pantip.com/topic/40410662.com"
label, pred = predict_url(url_test)
print("Label:", label)
print("Probabilities:", pred)


ModuleNotFoundError: No module named 'numpy._core'

In [ ]:
# --- Inlined utils (converted from phish_detector/utils.py) ---
import re
import math
from collections import Counter
from urllib.parse import urlparse

def parse_host_and_scheme(url):
    try:
        p = urlparse(url if '://' in url else 'http://' + url)
        return p.hostname or '', p.scheme or 'http'
    except:
        return '', ''

def is_ip_host(host):
    return bool(re.match(r'^\d+\.\d+\.\d+\.\d+$', host or ''))

def count_subdomains(host):
    if not host: return 0
    parts = host.split('.')
    return max(0, len(parts) - 2)

def has_double_slash_in_path(url):
    return '//' in urlparse(url if '://' in url else 'http://' + url).path

def has_tld_in_path(url):
    tlds = ['.com', '.net', '.org', '.co', '.io', '.gov']
    path = urlparse(url if '://' in url else 'http://' + url).path.lower()
    return any(t in path for t in tlds)

def has_symbols_in_domain(host):
    return bool(re.search(r'[^a-zA-Z0-9.-]', host or ''))

def domain_prefix_suffix_like_brand(host):
    return '-' in (host or '')

def brand_in_path_or_subdomain(host, url):
    brands = ['facebook','google','paypal','apple','microsoft','amazon','bank']
    path = urlparse(url if '://' in url else 'http://' + url).path.lower()
    subdomain = (host or '').split('.')[0].lower()
    return any(b in path or b in subdomain for b in brands)

def extract_html_features(html):
    hrefs = re.findall(r'href=["\'](.*?)["\']', html or '', flags=re.IGNORECASE)
    imgs = re.findall(r'<img[^>]+src=["\'](.*?)["\']', html or '', flags=re.IGNORECASE)
    scripts = re.findall(r'<script[^>]+src=["\'](.*?)["\']', html or '', flags=re.IGNORECASE)
    links_tag = re.findall(r'<link[^>]+href=["\'](.*?)["\']', html or '', flags=re.IGNORECASE)
    forms = re.findall(r'<form[^>]+action=["\'](.*?)["\']', html or '', flags=re.IGNORECASE)
    meta_keywords = re.findall(r'<meta[^>]+name=["\']keywords["\'][^>]+content=["\'](.*?)["\']', html or '', flags=re.IGNORECASE)
    return {
        'hrefs': hrefs,
        'imgs': imgs,
        'scripts': scripts,
        'links_tag': links_tag,
        'forms': forms,
        'meta_keywords': meta_keywords
    }

def abnormal_links(hrefs):
    for h in hrefs:
        if h.strip().lower().startswith(('javascript:','mailto:','data:')):
            return True
    return False

def forms_action_abnormal(forms, host):
    for a in forms:
        if a and host not in a and not a.startswith('/') and not a.startswith('#'):
            return True
    return False

def anchors_point_elsewhere(hrefs, host):
    count = 0
    total = max(1, len(hrefs))
    for h in hrefs:
        if host and host not in h and h.startswith('http'):
            count += 1
    return (count / total) > 0.5

def meta_keyword_mismatch(meta_keywords, host):
    if not meta_keywords: return False
    for kw in meta_keywords:
        if host.split('.')[0] not in kw:
            return True
    return False

# --- Extra Features ---
def digit_count(url):
    return sum(c.isdigit() for c in url)

def url_length(url):
    return len(url)

def url_entropy(url):
    prob = [freq/len(url) for freq in Counter(url).values()]
    return -sum(p * math.log2(p) for p in prob)


In [ ]:
import requests

def fetch_html(url):
    try:
        r = requests.get(url, timeout=5)
        return r.text
    except Exception as e:
        print("ไม่สามารถโหลดเว็บได้:", e)
        return ""

def phishing_score(url: str, html: str):
    host, scheme = parse_host_and_scheme(url)
    features = extract_html_features(html)
    score = 0
    reasons = []

    if is_ip_host(host):
        score += 2
        reasons.append("Host เป็น IP address")
    if count_subdomains(host) > 2:
        score += 1
        reasons.append("มี subdomain เยอะ")
    if has_symbols_in_domain(host):
        score += 1
        reasons.append("มี symbol แปลกใน domain")
    if domain_prefix_suffix_like_brand(host):
        score += 2
        reasons.append("ชื่อ domain คล้าย brand แต่มี -")
    if brand_in_path_or_subdomain(host, url):
        score += 1
        reasons.append("มี brand keyword ใน path หรือ subdomain")
    if has_double_slash_in_path(url):
        score += 1
        reasons.append("path มี double slash")
    if has_tld_in_path(url):
        score += 1
        reasons.append("TLD ปรากฏใน path")
    if abnormal_links(features['hrefs']):
        score += 1
        reasons.append("มีลิงก์ผิดปกติ")
    if forms_action_abnormal(features['forms'], host):
        score += 2
        reasons.append("form action ผิดปกติ")
    if anchors_point_elsewhere(features['hrefs'], host):
        score += 1
        reasons.append("anchor ชี้ไปเว็บอื่นเยอะ")
    if meta_keyword_mismatch(features['meta_keywords'], host):
        score += 1
        reasons.append("meta keywords ไม่ตรงกับ host")

    #new features
    dcount = digit_count(url)
    ulen = url_length(url)
    uentropy = url_entropy(url)

    if dcount > 5:
        score += 1
        reasons.append(f"มีตัวเลขเยอะ (digits={dcount})")
    if ulen > 75:
        score += 1
        reasons.append(f"URL ยาวผิดปกติ (length={ulen})")
    if uentropy > 4.0:
        score += 1
        reasons.append(f"Entropy ของ URL สูง (entropy={uentropy:.2f})")

    # add into features dict
    features.update({
        "digit_count": dcount,
        "url_length": ulen,
        "url_entropy": uentropy
    })

    return score, reasons, features, host, scheme


In [ ]:
from openai import OpenAI
from getpass import getpass

# Secure API key

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

🔑 Enter your OpenAI API key: ··········


In [ ]:
def prepare_llm_prompt(url, host, scheme, bilstm_result, bilstm_prob, score, reasons, features):
    return f"""
ค่าที่model BiLSTM วิเคราะห์จาก URL: {bilstm_result} (probability={bilstm_prob:.2f})

โดยช่วยคิดจาก feature ที่ดึงมาได้ดังนี้:
- Heuristic score: {score}
- Reasons: {', '.join(reasons)}

Features extracted:
- Digit count: {features.get('digit_count')}
- URL length: {features.get('url_length')}
- URL entropy: {features.get('url_entropy'):.2f}
- Hrefs: {features['hrefs']}
- Images: {features['imgs']}
- Scripts: {features['scripts']}
- Links tag: {features['links_tag']}
- Forms: {features['forms']}
- Meta keywords: {features['meta_keywords']}

โปรดให้เหตุผลสั้น ๆ และสรุปว่า 'Likely Phishing' หรือ 'Likely Safe'
"""

def analyze_with_llm(prompt: str):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
# สมมติ bilstm_prob เป็น array เช่น [0.2, 0.8] แสดง probability ของ [Safe, Phishing]
# เราเลือก class index ตามผลของ bilstm_result
import numpy as np

def run_analysis_on_url(url, call_llm=True):
    html = fetch_html(url)
    score, reasons, features, host, scheme = phishing_score(url, html)

    # เรียก BiLSTM predict
    bilstm_result, bilstm_prob_array = predict_url(url)  # ฟังก์ชันของคุณ
    # แปลงเป็น float scalar: เลือก class ตามผล
    class_idx = 0 if bilstm_result == "Likely Safe" else 1
    bilstm_prob = float(np.array(bilstm_prob_array)[class_idx])

    if call_llm:
        prompt = prepare_llm_prompt(url, host, scheme, bilstm_result, bilstm_prob, score, reasons, features)
        result = analyze_with_llm(prompt)
        print("\n------ Phishing Analysis (LLM) ------")
        print(result)
    else:
        result = None
        print("\n------ Rule-based Analysis Only ------")
        print(f"Rule-based score: {score}")
        print(f"Reasons: {', '.join(reasons)}")

    return {
        "result": result,
        "score": score,
        "reasons": reasons,
        "features": features,
        "host": host,
        "scheme": scheme
    }



In [ ]:
res = run_analysis_on_url("http://mycourses.ict.mahidol.ac.th", call_llm=True)
res

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

------ Phishing Analysis (LLM) ------
จากการวิเคราะห์ข้อมูลที่ให้มา เราสามารถสรุปได้ดังนี้:

1. **Heuristic Score**: ค่าคะแนน 2 แสดงว่ามีความเสี่ยงอยู่บ้าง แต่ไม่สูงมาก
2. **Reasons**: 
   - มี subdomain เยอะ: การมี subdomain หลายตัวอาจทำให้เกิดความสับสนและอาจเป็นสัญญาณของเว็บไซต์ที่ไม่ปลอดภัย
   - Meta keywords ไม่ตรงกับ host: การที่ meta keywords ไม่ตรงกับ host อาจบ่งบอกถึงการพยายามหลอกลวงผู้ใช้

3. **Features**:
   - URL length: 34 ตัวอักษร ถือว่าไม่ยาวเกินไป
   - URL entropy: 3.98 แสดงถึงความซับซ้อนของ URL ที่ไม่สูงมาก
   - Hrefs, Images, Scripts: มีการเชื่อมโยงไปยังเว็บไซต์ที่ดูเหมือนจะเป็นของจริง (mycourses.ict.mahidol.ac.th) ซึ่งเป็นสัญญาณที่ดี
   - Forms: มีฟอร์มสำหรับการเข้าสู่ระบบ ซึ่งเป็นฟีเจอร์ที่พบได้ในเว็บไซต์ที่ถูกต้อง

**สรุป**: แม้ว่าจะมีสัญญาณบางอย่างที่อาจบ่งบอกถึงความเสี่ยง (เช่น subdomain เยอะและ meta keywords ไม่ตรง) แต่โดยรวมแล้ว URL นี้มีลักษณะที่ดูเหมือนจะเป็นเว็บไซต์ที่ถูกต้องและปลอดภัย

**คำตอบ**: Likely Safe


{'result': 'จากการวิเคราะห์ข้อมูลที่ให้มา เราสามารถสรุปได้ดังนี้:\n\n1. **Heuristic Score**: ค่าคะแนน 2 แสดงว่ามีความเสี่ยงอยู่บ้าง แต่ไม่สูงมาก\n2. **Reasons**: \n   - มี subdomain เยอะ: การมี subdomain หลายตัวอาจทำให้เกิดความสับสนและอาจเป็นสัญญาณของเว็บไซต์ที่ไม่ปลอดภัย\n   - Meta keywords ไม่ตรงกับ host: การที่ meta keywords ไม่ตรงกับ host อาจบ่งบอกถึงการพยายามหลอกลวงผู้ใช้\n\n3. **Features**:\n   - URL length: 34 ตัวอักษร ถือว่าไม่ยาวเกินไป\n   - URL entropy: 3.98 แสดงถึงความซับซ้อนของ URL ที่ไม่สูงมาก\n   - Hrefs, Images, Scripts: มีการเชื่อมโยงไปยังเว็บไซต์ที่ดูเหมือนจะเป็นของจริง (mycourses.ict.mahidol.ac.th) ซึ่งเป็นสัญญาณที่ดี\n   - Forms: มีฟอร์มสำหรับการเข้าสู่ระบบ ซึ่งเป็นฟีเจอร์ที่พบได้ในเว็บไซต์ที่ถูกต้อง\n\n**สรุป**: แม้ว่าจะมีสัญญาณบางอย่างที่อาจบ่งบอกถึงความเสี่ยง (เช่น subdomain เยอะและ meta keywords ไม่ตรง) แต่โดยรวมแล้ว URL นี้มีลักษณะที่ดูเหมือนจะเป็นเว็บไซต์ที่ถูกต้องและปลอดภัย\n\n**คำตอบ**: Likely Safe',
 'score': 2,
 'reasons': ['มี subdomain เยอะ', 'meta keywor